# Estructuras de Datos - Universidad Nacional de Tres de Febrero 
# Trabajo Práctico Nº 2 - Indexador de Noticias

### Lottero Bruno - Leg. 18434

### Dependencias
- configparser
- requests
- beautifulsoup4
- lxml

Se pueden instalar con: pip3 install -r requirements

### A continuación se describen las clases y decisiones de diseño más importantes

## NewsReader.py
#### Esta clase contiene los métodos para leer y almacenar los artículos
#### Contiene métodos para:
- Crear los directorios/archivos xml destino si no existen
- Recolectar y almacenar los artículos según los medios/canales indicados en la configuración

### Decisiones de diseño:
- Para evitar almacenar noticias en un canal, se verifica por xpath que no exista previamente en el mismo archivo XML
- Se utilizó el paquete requests para hacer las peticiones HTTP correspondientes

### Por ejemplo
Leemos la configuración para tests

In [10]:
import configparser

config = configparser.ConfigParser()
config.read('tests/config.ini', encoding='utf-8')

['tests/config.ini']

También definimos una función callback para que nos imprima mensajes por consola (más adelante se explica en más detalle)

In [13]:
def print_callback(message, *args):
    """
    Función callback para recibir mensajes e imprimir un texto adecuado por consola
    """
    messages = {
        "DLERR": "No se pudo descargar el XML de %s",
        "PARSEERR": "No se pudo parsear el XML de %s",
        "NEWARTICLE": "Agregado %s - %s - %s",
        "BADFORMAT": "Mal formato de título o fecha, salteando...",
        "WAITING": "Esperando %s segundos para reiniciar descargas",
        "CANINTERR": "Si lo desea presione CTRL+C para cancelar y volver al menú principal",
        "BLKNE": "El archivo intermedio %s no existe, salteando...",
        "INDERR": "No se puede indexar el artículo con título %s",
        "XMLNF": "No existe el archivo XML %s"
    }
    print(messages.get(message) % args)

Construímos NewsReader

In [14]:
from NewsReader import NewsReader

news_reader = NewsReader(config, callback=print_callback)
news_reader.collect_news()

Agregado TELAM - política - Alberto Fernández: &quot;Fue un gran presidente al que le tocó un tiempo difícil&quot;
Agregado TELAM - política - Guerra judicial entre los dos principales candidatos a gobernador de Salta
Agregado TELAM - política - El bloque peronista se reunió por primera vez luego del triunfo electoral
Agregado TELAM - política - Palacios: Cuando tuve a la Virgen conmigo recordé a los caídos y mis compañeros
Agregado TELAM - política - Ordenan a Vialidad que brinde datos de contratos sobre corredores viales
Agregado TELAM - política - Fernández: &quot;Los argentinos están cansados de levantarse cada mañana para ver cuál es la tasa&quot;
Agregado TELAM - política - Archivan una denuncia contra Pichetto por sus dichos sobre la villa 1-11-14
Agregado TELAM - política - Michetti y Cristina Fernández van a reunirse en noviembre para facilitar la transición
Agregado TELAM - política - Rodríguez Larreta se reunió con Tombolini y ahora buscará encontrarse con Lammens
Agregado

In [ ]:
Luego de esto deberíamos tener en tests/TELAM/ los xml de economía y política

## Index.py
#### Es la clase que representa al índice invertido
#### Contiene métodos para:
- Procesar los cuerpos de noticias como bloques (cada medio es un bloque)
- Combinar los índices intermedios en un solo índice

### Decisiones de diseño:
- El formato de los índices intermedios consiste en palabras de longitud variable:
    - 4 bytes de term_id + 4 bytes para indicar tamaño de doc_list + n bytes de doc_list
- El formato de la lista de apariciones final consiste solamente en concatenar cada uno de los doc_list de los intermedios, ya que la metadada se encuentra en el diccionario almacenado con pickle